In [1]:
# clone colab
!git clone https://github.com/jchen6727/colab.git

Cloning into 'colab'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 44 (delta 16), reused 32 (delta 8), pack-reused 0
Receiving objects: 100% (44/44), 1.20 MiB | 8.55 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [8]:
# import and filestrings
import pandas
import plotly.express as px

import argparse

bfiles = ['colab/bopt/b{}.csv'.format(i) for i in ['011', '101', '110', '111']]
ofiles = ['colab/optuna/o{}.csv'.format(i) for i in ['011', '101', '110', '111']]

print(bfiles)
print(ofiles)


In [11]:
# load csv to pandas
df = pandas.read_csv('colab/bopt/b011.csv')

In [18]:
# create additional columns (min)
df['min_loss'] = df['loss'].expanding().min()

actual = { # weights from cfg, AMPA, GABA, NMDA
    'PYR->BC_AMPA' : 0.36e-3, "BC->BC_GABA"  : 4.5e-3 , "PYR->BC_NMDA" : 1.38e-3 ,
    'PYR->OLM_AMPA': 0.36e-3, "BC->PYR_GABA" : 0.72e-3, "PYR->OLM_NMDA": 0.7e-3  ,
    'PYR->PYR_AMPA': 0.02e-3, "OLM->PYR_GABA": 72e-3  , "PYR->PYR_NMDA": 0.004e-3,
    'PYR': 3.34,
    'BC': 19.7,
    'OLM': 3.47
}

rename = {
       'config/netParams.connParams.BC->BC_GABA.weight',
       'config/netParams.connParams.BC->PYR_GABA.weight',
       'config/netParams.connParams.OLM->PYR_GABA.weight',
       'config/netParams.connParams.PYR->BC_AMPA.weight',
       'config/netParams.connParams.PYR->BC_NMDA.weight',
       'config/netParams.connParams.PYR->OLM_AMPA.weight',
       'config/netParams.connParams.PYR->OLM_NMDA.weight',
       'config/netParams.connParams.PYR->PYR_AMPA.weight',
       'config/netParams.connParams.PYR->PYR_NMDA.weight',
}

rename = { n: n.split('.')[2] for n in rename if n in df.columns}

params = rename.values()

df.rename(rename, axis=1, inplace=True)

def expanding_idxmin(df, col):
    curr = 0
    for i in range(len(df)):
        if df[col][i] < df[col][0:i].min():
            yield i
            curr = i
        else:
            yield curr

for param in params:
    df['MIN_{}'.format(param)] = df[param][expanding_idxmin(df, 'loss')].reset_index(drop=True)

for param in ['PYR', 'BC', 'OLM']:
    df['MIN_{}'.format(param)] = df[param][expanding_idxmin(df, 'loss')].reset_index(drop=True)


In [7]:
# show plots
fig = px.scatter(df, x=df.index, y='loss')
fig.add_scatter(x=df.index, y=df['min_loss'], mode='lines', name='min_loss')
fig.show()
################################################################################
#
#  LOSS (MSE) PER TRIAL RUN
#
################################################################################

In [19]:
# show plots
fig = px.scatter(df, x=df.index, y='PYR')
fig.add_scatter(x=df.index, y=df['MIN_PYR'], mode='lines', name='MIN_PYR')
fig.add_hline(actual['PYR'])
fig.show()
################################################################################
#
#  LOSS (MSE) PER TRIAL RUN
#
################################################################################

In [20]:
# show plots
fig = px.scatter(df, x=df.index, y='BC')
fig.add_scatter(x=df.index, y=df['MIN_BC'], mode='lines', name='MIN_BC')
fig.add_hline(actual['BC'])
fig.show()
################################################################################
#
#  LOSS (MSE) PER TRIAL RUN
#
################################################################################


In [21]:
# show plots
fig = px.scatter(df, x=df.index, y='OLM')
fig.add_scatter(x=df.index, y=df['MIN_OLM'], mode='lines', name='MIN_OLM')
fig.add_hline(actual['OLM'])
fig.show()
################################################################################
#
#  LOSS (MSE) PER TRIAL RUN
#
################################################################################